# Описание очистки датасета

In [1]:
import pandas as pd

In [2]:
import sys

# setting path
sys.path.append("../src/data")

import data_cleanser

MlflowException: API request to http://locahost:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='locahost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=cleaning_dataset (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000279AE6BB350>: Failed to establish a new connection: [Errno 11002] getaddrinfo failed'))

In [3]:
df = pd.read_csv("../data/raw/GoodReads_100k_books.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   author        100000 non-null  object 
 1   bookformat    96772 non-null   object 
 2   desc          93228 non-null   object 
 3   genre         89533 non-null   object 
 4   img           96955 non-null   object 
 5   isbn          85518 non-null   object 
 6   isbn13        88565 non-null   object 
 7   link          100000 non-null  object 
 8   pages         100000 non-null  int64  
 9   rating        100000 non-null  float64
 10  reviews       100000 non-null  int64  
 11  title         99999 non-null   object 
 12  totalratings  100000 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 9.9+ MB


In [4]:
df.head()

,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


Необходимо удалить все пропуски в колонках `desc`, `genre`, `author` и `title`. Книги без описания, жанровой принадлежности, автора или названия не помогут в построении рекомендательной системы.

In [5]:
df = data_cleanser.drop_important_nans(df)

Amount of rows with Nans dropped = 14515


Книги у которых одинаковое сочетание автор/название. Дублируют уже известную информацию. Стоит их удалить. 

In [6]:
df = data_cleanser.drop_repeated_books(df)

Amount of duplicated rows dropped = 39


Столбцы с форматом книги, количеством обзоров на книгу и количеством страниц не нужны для построения рекомендательной системы.

In [7]:
df = data_cleanser.drop_useless_columns(df)

Names of dropped columns: bookformat, reviews, pages


Часть значений датафрейма составляют книги написанные на не-английском языке. Рекомендательная система рассчитана только на англоязычные книги. Все книги с нелатинскими символами удаляются из датафрейма.

In [8]:
df = data_cleanser.drop_non_english_books(df)

Amount of rows containing non-english letters dropped = 
        36720


Результат очистки:

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85485 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   author        85485 non-null  object 
 1   bookformat    84054 non-null  object 
 2   desc          85485 non-null  object 
 3   genre         85485 non-null  object 
 4   img           84851 non-null  object 
 5   isbn          74141 non-null  object 
 6   isbn13        76919 non-null  object 
 7   link          85485 non-null  object 
 8   pages         85485 non-null  int64  
 9   rating        85485 non-null  float64
 10  reviews       85485 non-null  int64  
 11  title         85485 non-null  object 
 12  totalratings  85485 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 9.1+ MB


In [10]:
df.head()

,author,desc,genre,img,isbn,isbn13,link,rating,title,totalratings
0,Laurence M. Hauptman,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,3.52,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,4.51,Fashion Sourcebook 1920s,41
2,Andy Anderson,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,4.15,Hungary 56,26
3,Carlotta R. Anderson,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,3.83,All-American Anarchist: Joseph A. Labadie and ...,6
6,Jeffrey Pfeffer,"""Competitive Advantage Through People"" explore...","Business,Leadership,Business,Management,Romanc...",https://i.gr-assets.com/images/S/compressed.ph...,087584717X,9.78E+12,https://goodreads.com/book/show/1001092.Compet...,3.65,Competitive Advantage Through People: Unleashi...,20


# Генерация текстовых фич

In [2]:
import pandas as pd

In [3]:
import sys

# setting path
sys.path.append("../src/features")

import text_features_generator

In [4]:
df = pd.read_parquet("../data/interim/GoodReads_books_clean.parquet")

Можно сгенерировать дополненное текстовое описание книги добавив в него жанр и оценку читателей

In [5]:
text_features_generator.generate_extended_description(df)

0        This book belongs to the following genres: His...
1        This book belongs to the following genres: Cou...
2        This book belongs to the following genres: Pol...
3        This book belongs to the following genres: Lab...
4        This book belongs to the following genres: Bus...
                               ...                        
48721    This book belongs to the following genres: Non...
48722    This book belongs to the following genres: Ani...
48723    This book belongs to the following genres: Chr...
48724    This book belongs to the following genres: Non...
48725    This book belongs to the following genres: Fan...
Length: 48726, dtype: object

Чтобы сделать исчерпывающее текстовое описание можно добавить еще название и автора.

In [6]:
text_features_generator.generate_maximally_extended_description(df)

0        The book is titled Between Two Fires: American...
1        The book is titled Fashion Sourcebook 1920s by...
2        The book is titled Hungary 56 by Andy Anderson...
3        The book is titled All-American Anarchist: Jos...
4        The book is titled Competitive Advantage Throu...
                               ...                        
48721    The book is titled The Sea Inside by Philip Ho...
48722    The book is titled A Horse for Angel by Sarah ...
48723    The book is titled A Faith Worth Sharing: A Li...
48724    The book is titled A Volcano Beneath the Snow:...
48725    The book is titled Paranormal Nation: Why Amer...
Length: 48726, dtype: object